Chapter 3.  Implication
=======

An implication such as $P \Rightarrow Q$ has an antecedent ($P$) and a consequent ($Q$).  If the antecedent is true, the consequent must also be true.  **Prove-It** has an `Implies` **operation** (a sub-class of **Expression**) that may be used to represent an implication, formatted with the $\Rightarrow$ symbol.  While this is a concept used in the core, `Implies` is actually defined outside of the core in the `proveit.logic` package.  It is known in the core for use in the *modus ponens* and *hypothetical reasoning* derivation steps discussed below.

Below we import some helpful and necessary elements, define the `Implies` object $A \Rightarrow B$, and then take a brief tour of the `Implies` class:

In [1]:
from proveit._common_ import A, B, C, X
from proveit.logic import Implies 
%begin implication

In [2]:
A_impl_B = Implies(A, B)

A_impl_B: A => B

Implies is an `Operation` (a sub-class of `Expression`):

In [3]:
A_impl_B.exprInfo()

,core type,sub-expressions,expression
0,Operation,operator: 1operands: 2,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAEAAAAAPBAMAAACmSdRUAAAAMFBMVEX///92dnYQEBBERERUVFS6 urqIiIhmZmaqqqrc3Nzu7u6YmJgyMjIiIiLMzMwAAACFkkdwAAAAAXRSTlMAQObYZgAAAOhJREFU GNNjYACBtwxYwJb/Li42ARD2eWwKGJQZGNi/glnc9xcghOFMvm9A/AXMnFE/AaHgFIzB9hOo9SOY mZAfgFAQAmMwKzAw8GqBWKwL+jcgFDAmQBn8Dgych8EWLmV4DxZkvwsG36EK3rul1UI0BjDML0By PMdRCJ0/gYHrCti1Li7+BsjeW6MKpoxAikBGsDIwMB1Akuc7CPEpyIf7BYBEA9CxCkhuuAyR5/sN JPyBkpxAzPMByQkOiGDg/AOyEeTlXwgFc5CC4REw1Cbrn2Tgtv+D8IYlhGK3/+LicxpbFD1A5QIA YgVCLvPPxacAAAAASUVORK5CYII= "" style=""display:inline;vertical-align:middle;"" />"
1,Literal,,
2,ExprTuple,"3, 4","<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAADUAAAAVBAMAAAAUZLt1AAAAMFBMVEX////c3NwiIiJERESqqqqY mJh2dnZmZmYQEBDu7u4yMjJUVFTMzMyIiIi6uroAAAAs73DqAAAAAXRSTlMAQObYZgAAATZJREFU KM9jYMAH+C5gFY4DEach7EMwQdb9v3dvXsfAfgDIXgkRWg/XwWHAwFD/gGEBAwMvxEhW/QkwufNA kfMbGMKAqiBiIvUCMLn7QFZ8AsMMBoZXEIEL9x/A5CyAWGMCA7MDQxnEsRP8A2ByvxkYpCwZGNgK GJaC+fPAtoABz++711WBNMsChj6wwAMG+QKoHO9HBoZJG4BGNTD8A6vdvXu/AVSOuQEo/xUo9gEi xwd07wKoHD9QD+s3sFwLiO/AwMDUAPMe0FHcX8BmmgK5jEA5rg8MDBNg3pMHGsK6gCEdyJ0FsuYn A+cnqPd4uiaA/fCIgUG4fyEDq/2vAs5+oJbi/7t3rwL5lesBA4cAUnQhsUFhxpqA4LIiR2AuIo7Q 5ThBfpJG8JFTADfIfEbsaeIOAwMABOxNRsfkiy0AAAAASUVORK5CYII= "" style=""display:inline;vertical-align:middle;"" />"
3,Variable,,
4,Variable,,


We can access the `antecedent` and the `consequent` of the implication as follows:

In [4]:
A_impl_B.antecedent

A

In [5]:
A_impl_B.consequent

B

And we can take a peak at the instance attributes and methods:

In [6]:
dir(A_impl_B)

['EquivalenceClass',
 'SequenceClass',
 'StrongRelationClass',
 'WeakRelationClass',
 '_RelationClasses',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_checkRelabelMap',
 '_checkedStrongRelationClass',
 '_checkedWeakRelationClass',
 '_class_path',
 '_clear_',
 '_config_latex_tool',
 '_coreInfo',
 '_equiv_method_evaluate_',
 '_equiv_method_evaluated_',
 '_equiv_method_evaluation_',
 '_equiv_method_simplification_',
 '_equiv_method_simplified_',
 '_equiv_method_simplify_',
 '_extractCoreInfo',
 '_extractExprClass',
 '_extractInitArgs',
 '_extractMyInitArgs',
 '_extractReferencedObjIds',
 '_extractStyle',
 '_fixedTransitivitySort',
 '_formatted',
 '_formattedOperation

## Modus Ponens

*Modus ponens* is a straightforward derivation step in which you can derive $B$ assuming that $A$ and $A \Rightarrow B$ are both true statements.  You can apply this derivation step explicitly by calling the `deriveConsequent` method of any `Implies` object.

In [7]:
B_from_A = A_impl_B.deriveConsequent(assumptions=[A, A_impl_B])

B_from_A: {A , A => B} |- B

Recall from the previous tutorial chapter that **Prove-It**, in its core, is not concerned about truth-aptness.  Neither $A$ nor $B$ are required to be intrinsically truth-apt.  The above **known truth** simply means that $B$ is a true statement *if* we assume that $A$ and $A \Rightarrow B$ are true statements. The associated proof makes explicit the application of modus ponens:

In [8]:
B_from_A.proof()

,step type,requirements,statement
0,modus ponens,"1, 2","<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAGsAAAAVBAMAAACzuQ2OAAAAMFBMVEX///9ERETc3NyqqqpUVFS6 urrMzMwyMjIQEBCIiIh2dnYiIiLu7u5mZmaYmJgAAABJJ2g3AAAAAXRSTlMAQObYZgAAAaBJREFU OMtjYGDgu8dACmBbpQCieHYxkAbYmkEk5wcobwOSVDY29U/+u7iIBzAwXAdxuBogguwfkFTUY7Wn kYGB9ysDwwpkba//IOT59h+As3kQrvsJxF9QtTHZfkJoU76vAGezw03gBKrg+4WqjZPzK0LbhPkB cDarA4zFCFTJ3YOqLYb5H0LlAfsHCDNWwhj8DgxMhQeg2ngLwD4IALkcCo4y5E8A0bd3g8D+C1Dh fM+Zt0HGvQdxMsDuYWVg+gvXFsCgfwEpAM2hLp6vwMCzBeRboGRKG1goxcXlIzzEXFz8BZC08VRA 9EmAtIKsEwcSEQkQCxja4V5jYOAoQNKWvgZCfwE5cAEwWICYgQWkghmId8CUGQBlGhB+2y0DdQQo zPwNoH7jAiUqZUjKAEctkwFqkjFBijam7wxIEcADslH+AMN5kBHHQVGESDJMBUjRllGDpI1J4rsC w8P/tQwngAYq9hcx8Ml/hwclIzTB8Mp/cXGvY2BAS5NgEIQln2CIYGp7QEyGA2vjI1kbOL/x7EQI cBKVu1shmWsx6WUJANQzceylMolFAAAAAElFTkSuQmCC "" style=""display:inline;vertical-align:middle;"" /> ⊢"
1,assumption,,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAFMAAAAVBAMAAADInc9CAAAAMFBMVEX///9ERETc3NyqqqpUVFS6 urrMzMwyMjIQEBCIiIh2dnYiIiLu7u5mZmaYmJgAAABJJ2g3AAAAAXRSTlMAQObYZgAAAU9JREFU KM9jYGDgu8dACLCtUgBRPLsYCAO2ZhDJ+QHK24BFCZ//FxfHUiDjOojH1QARZf+AzTgOAQaG+wEM DCuQlb7+g1DAA2flTwBiB1SlTLafEErZD8BY84H+eb8AVSkn51eEUlYHGEsCiHccQFUaw/wPyYkr YYwvDAzqMgxQpbwFYNcFsP0Ey93eDQL7L0CD5cvMuftAjPcgIiMAbCUD019kn5sHQJwF9MBpkGvY gXpT2sBiKS4uH5GV8lSA1TICXccJdpn4AwaGiAQQCyjRjqw0fQ2Y4geayPcLyOAGhgIDC8itzCCP Irl1tww0WIFmsXyGuZULFKHKQFyPZKgJUrDqFzAgBRYPiCcPD3cGpgJEsLK1H0BSyiTxXYHh4f9a uFJGBQh98b+LS/kDBga0NICS5jBEcCplwK6Ujyil4PTKs5OYXNAKSemLictbAOAEWqBdr+e5AAAA AElFTkSuQmCC "" style=""display:inline;vertical-align:middle;"" /> ⊢ <img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAEAAAAAPBAMAAACmSdRUAAAAMFBMVEX///92dnYQEBBERERUVFS6 urqIiIhmZmaqqqrc3Nzu7u6YmJgyMjIiIiLMzMwAAACFkkdwAAAAAXRSTlMAQObYZgAAAOhJREFU GNNjYACBtwxYwJb/Li42ARD2eWwKGJQZGNi/glnc9xcghOFMvm9A/AXMnFE/AaHgFIzB9hOo9SOY mZAfgFAQAmMwKzAw8GqBWKwL+jcgFDAmQBn8Dgych8EWLmV4DxZkvwsG36EK3rul1UI0BjDML0By PMdRCJ0/gYHrCti1Li7+BsjeW6MKpoxAikBGsDIwMB1Akuc7CPEpyIf7BYBEA9CxCkhuuAyR5/sN JPyBkpxAzPMByQkOiGDg/AOyEeTlXwgFc5CC4REw1Cbrn2Tgtv+D8IYlhGK3/+LicxpbFD1A5QIA YgVCLvPPxacAAAAASUVORK5CYII= "" style=""display:inline;vertical-align:middle;"" />"
2,assumption,,⊢


In order for the *modus ponens* derivation step to succeed, the implication and the hypothesis must be proven to be true or appear under the applicable assumptions. Below, we see errors in the attempted proof if we omit either of the assumptions $A$ or $A \Rightarrow B$:

In [9]:
from proveit import ModusPonensFailure
try:
    A_impl_B.deriveConsequent(assumptions=[A])
    assert False, "Expecting a ModusPonensFailure error; should not make it to this point"
except ModusPonensFailure as e:
    print("EXPECTED ERROR:", e)

EXPECTED ERROR: Unable to prove B assuming {A}: Implication, A => B, is not proven


In [10]:
from proveit import ModusPonensFailure
try:
    A_impl_B.deriveConsequent(assumptions=[A_impl_B])
    assert False, "Expecting an ModusPonensFailure error; should not make it to this point"
except ModusPonensFailure as e:
    print("EXPECTED ERROR:", e)

EXPECTED ERROR: Unable to prove B assuming {A => B}: Antecedent of A => B is not proven


When a **known truth** wraps an **implication**, the **assumptions** of the **known truth** are automatically added.  This is the case for any `Expression` method that accepts an `assumptions` argument and is called indirectly through a `KnownTruth` object that wraps the `Expression` object.  We demonstrate this in the following two cells:

In [11]:
A_impl_B_truth = A_impl_B.prove([A_impl_B])

A_impl_B_truth: {A => B} |- A => B

In [12]:
# A => B is automatically included as an assumption because it is 
# an assumption of A_impl_B_truth
A_impl_B_truth.deriveConsequent(assumptions=[A])

{A , A => B} |- B

### Using an overcomplete set of assumptions

When taking any derivation step under a set of assumptions, any assumptions that are unnecessary will be discarded.  In the example below, we include $C$ as an extra, irrelevant assumption.  It is discarded as it is not needed in the proof.

In [13]:
A_impl_B.deriveConsequent(assumptions=[A_impl_B, A, C]) 

{A => B , A} |- B

## Hypothetical Reasoning

*Hypothothetical reasoning* is, in some sense, the reverse process of *modus ponens*.  In *modus ponens* the consequent is derived from an implication (and its antecedent).  In *hypothetical reasoning* the implication is derived from the consequent, discarding the antecedent as an assumption.  A common notation to indicate a derivation rule is to display a horizontal line with a new truth below the line that can be derived from what is above the line.  Using this notation, we have

Modus ponens: 
$\begin{array}{c}
\vdash A \Rightarrow B \\
\hline
\{A\} \vdash B
\end{array}$

Hypothetical reasoning: 
$\begin{array}{c}
\{A\} \vdash B \\
\hline
\vdash A \Rightarrow B
\end{array}$

We chose to write these in a form that exhibits the symmetry, though it does not matter whether something is an assumption or a prerequisite truth.  In our *modus ponens* example above, we actually had 

$\begin{array}{c}
\hline
\{A, A \Rightarrow B\} \vdash B
\end{array}$

with no prerequisites, only assumptions.

If $B$ is true assuming $A$, it follows, via *hypothetical reasoning*, that $A \Rightarrow B$.  We prove an implication by assuming the antecedent and deriving the consequent, reasoning through a hypothetical scenario.  This step may be taken by calling `asImplication` (or `asImpl` as an abbreviation) on a `KnownTruth` object.

We will demonstrate *hypothetical reasoning* by proving the transitivity property of implications, $A \Rightarrow C$ given $A \Rightarrow B$ and $B \Rightarrow C$.   We have already created the `A_impl_B` object to represent $A \Rightarrow B$ and have proven $\{A,A \Rightarrow B\} \boldsymbol{\vdash} B$.  Let us now create a `B_impl_C` object to represent $B \Rightarrow C$:

In [14]:
B_impl_C = Implies(B, C)

B_impl_C: B => C

Now we can prove $C$ given $A$, $A \Rightarrow B$, and $B \Rightarrow C$.  This uses the previously derived proof of $\{A,A \Rightarrow B\} \boldsymbol{\vdash} B$ and extends it by deriving the consequent of $B \Rightarrow C$ under appropriate assumptions. 

In [15]:
CviaTransitivity = B_impl_C.deriveConsequent([A, A_impl_B, B_impl_C])

CviaTransitivity: {A , A => B , B => C} |- C

We are now ready to apply *hypothetical reasoning* by calling `asImplication` on this `KnownTruth**:

In [16]:
A_impl_C = CviaTransitivity.asImplication(A)

A_impl_C: {A => B , B => C} |- A => C

Below is the full **proof** of this **known truth**:

In [17]:
A_impl_C.proof()

,step type,requirements,statement
0,hypothetical reasoning,1,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAJ0AAAAVBAMAAACu6/FQAAAAMFBMVEX///9ERETc3NyqqqpUVFS6 urrMzMwyMjIQEBCIiIh2dnYiIiLu7u5mZmaYmJgAAABJJ2g3AAAAAXRSTlMAQObYZgAAAf9JREFU OMtjYGDgu8dAFcC2SgFE8exioBJgawaRnB+gvA1YlPD5f3FxLMXGQQEnRe4+0GVguA5iczVAxNg/ YFPJIcDAcD8AGwcJcPUxMMQC8Qpk817/QSg4AGflTwBiB2wcBh44oxqonusTqnlMtp8Q5tXAWfOB Yfx+ATYOAzvMVvYeIMHagGoeJ+dXhHkhcJYEEO84gI3DwApz6X0DkHsuoJoXw/wPKUAmwFhfGBjU ZbByGBhWQiim72BPJ0DN4y0AcwPYfoJ5u8HgLywJfJk5dx8DBuc2WNH+C2AOL9wh70FEBjjCWBmY /iLFGDM0WXACA/U0zF8oHBAwB2vlhgc8O9CClDYwM8XF5SNyEjjeCqYYgYHBCbMfhQP2VAXIQP4P 8NgWf8DAEJEAYgEl2pGTehUk2PmBzuH7BRVE4YBA+hpwqgSbZwZyKSjuWUDhxwyKOaTw2wiNxXyg hSyfGdA5kPDbDYkbFnDCvQALPy5QPlMG4nqk4HNASnH6BdAEDuPAgAnUL6CI5AKpgKcXHpAqeUSu UEdKfmztBxh4fiNxYIAJZvR7AwbWSwxI5jFJfFdgePi/Fq5UCEpf/O/iUg70JU+/AoIDA4wKsMB2 973MgGweBkhAF2BVwFI8YYjgNA+z3CKqAASbx0c988DlH89OIlROIKp8huQBvsVUrD8AopSgUXB2 lz4AAAAASUVORK5CYII= "" style=""display:inline;vertical-align:middle;"" /> ⊢ <img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAD8AAAAPBAMAAAChCwpBAAAAMFBMVEX///92dnYQEBBERERUVFS6 urqIiIhmZmaqqqrc3Nzu7u6YmJgyMjIiIiLMzMwAAACFkkdwAAAAAXRSTlMAQObYZgAAAOFJREFU GNNjYACBtwxYAFeyezVnAYR9HpsCoQ0MrEUbwEzu+wsQ+mCMNbeAhPwEMHtG/QS4Ah6Y2kMgRhCE nZAfAFfA6gCh2X+AyOkQsQX9GxBWi0Go+QdA5DoweynD+wQQXXsXBO5DHF6PpCeAYX4Bkut7wPbd RziLz8XF3wDZ/4dAKv6D2WAXszIwMB1AUvBGCkTqgwiOByCygYGBVwHhhrtWYGX+IM3sIBYnUAHP B4QBLRAK7G6wWWuAmPkXXJ4Tahs3MBzCQQ6drH+Sgdv+D9wbzDDXPzlS3oAtivjQ+AD0rD2U4WZv AQAAAABJRU5ErkJggg== "" style=""display:inline;vertical-align:middle;"" />"
1,modus ponens,"2, 3","<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAALcAAAAVBAMAAAD2lGJJAAAAMFBMVEX///9ERETc3NyqqqpUVFS6 urrMzMwyMjIQEBCIiIh2dnYiIiLu7u5mZmaYmJgAAABJJ2g3AAAAAXRSTlMAQObYZgAAAjlJREFU OMu1VT1IHFEQHlfvztPdNaTRQnCDRaqghZDSKxbEIpyIhU3AQlCw8AwKEkKMSFIF7iytFNKoCCrY +AN3TdTO6xREvSK9YhSJmpiZ97dv915xhTew875vdvbbt/PmvQUA9xye3+KrHg32DlTD4nPkk1eC jmm3xkz5v558v7nPRHSz59+dW0WAYyINX0R0RsuYMc5mFsC5NRLNlksQOynhqIu7+YJK0DEoGP+D 142J4HwlONhDt+5FxNsvPJWqY/gkQfIaX3tvIgAJOYVpAoMQEc8tBhXUMQxIUIOZjd9NBCCW4qPz m/ybiHis0F1SmRrGhJwATSmwJgomgrbCh44M+X0p7jAKP2Fciijs7DL7K8LjvQun6qUBOWVJ+SIj Fyrhktwor0AfdBRlXMdotVN8XPTA3pLBECF7y3TyarESKDH8jS++76dfyEbQMO+Ar2xoIU05sxBh /TJN6k9BgzXj3cEhVmaA+owsuYbZyz7y2lLjXS6JYIiQjayRz7JPJclGulnHdDqRyd2ksKj5JteO /0OX7oQo4TXfbWXxNOU6quYNG+gsTEtc8e+RWJU8FXS2dSf2iyTKusQ6Uyewbwha8YA69xHshwAr e6119ijuqKMNjUizRB1d7PP3Xkj8VfYDuG13RTvrKayeeym2R9uN3/MZweG1RtQGk10yPHnGKxfa oXwtvfKTaCga6DedsOUHWJm4W8lRXaroQGfibpXE2Xlub2uRXAVPJSv7E/GN5/6o1j/0P+z2uKtH x+MYAAAAAElFTkSuQmCC "" style=""display:inline;vertical-align:middle;"" /> ⊢"
2,assumption,,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAFMAAAAVBAMAAADInc9CAAAAMFBMVEX////c3NxERESqqqpUVFS6 urrMzMwyMjIQEBCIiIh2dnYiIiLu7u5mZmaYmJgAAABXZE4RAAAAAXRSTlMAQObYZgAAAVZJREFU KM9jYGDgu8dACLCtEgBRPLsYCAO2ZhDJ+QFI8Pl/cXEqxabopMrdB7IMDNdBbK4GEMmhwMBwPwBT JVcfA0MsEK9AKM2fAMQOcBUHoDRPNZDF9RFF6Xygu98vgCutgdLsPUCCtQFFqQYQ7zgAVxoCpe8b AAnGCyhKvzAwiOsgOXECmGL8DnZFAlQpbwEoML7MnLsPLMu7Gwz+Qjj/YHrfg4gMkL85gS4/7YDk c2ZwyHF/hPHZgY5IaQOxmICu4PyHHErHW4EE/wewA0CE+gMGhogEkBjQRL5fyNFTBfIjB1ipGch8 UOiwgNyaD9TD8hnJrZvAocHyB0RegLmVawM0WOULkJwKcTfbT5ACkC5EYAGDla0dEawM4jCPGzCw XmRAVnrpv4tL+QMkpyrCnOzue5kBRSkGSEAXwK2UAatSPqKUgtMrz05ickErmOJbTFzeAgD9sl8j gQWWbAAAAABJRU5ErkJggg== "" style=""display:inline;vertical-align:middle;"" /> ⊢ <img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAEEAAAAPBAMAAABJi79qAAAAMFBMVEX///+qqqru7u4iIiJ2dnbc 3NyIiIjMzMxmZmYQEBC6uroyMjJERERUVFSYmJgAAADLUcuvAAAAAXRSTlMAQObYZgAAAPhJREFU GNNjYGB4+//u3dMLGDABU86NPtYGEGsOAwPHLywqPB8wcHU+ADLYvwHxX4ROGEPbCEj4BwAJtq8M DHxf4Cq4FaCMnSDGShCLZwIDA/MkuAquCxCa4yOIjAAR/BcYWPcoIKx3g1DxG0CkOoiov5bW/AAs 2GwMAvZg5zP0P4DryQ9gYLJF9oM02Ov2AXCBkyBVD5DDYSdIyX8wG2w7yKPvHZBUVDuByPkggrEA FBz/gMR9AYQ7jA+B1d0HaedggAYH628kI2QhVH0CkHCCBUflJoQC1g0Qmg8YHitAruU4//fu7X1I RvDA/FC7t0OAARtgR+MDAAhbPiSyASIoAAAAAElFTkSuQmCC "" style=""display:inline;vertical-align:middle;"" />"
3,modus 

Note that we can take any of the assumptions to be the antecedent and then that assumption will be eliminated:

In [18]:
A_impl_B__impl__C = CviaTransitivity.asImplication(A_impl_B)

A_impl_B__impl__C: {A , B => C} |- (A => B) => C

In [19]:
A_impl_B__impl__C.proof()

,step type,requirements,statement
0,hypothetical reasoning,1,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAG0AAAAVBAMAAAC+p33JAAAAMFBMVEX///9ERETc3NyqqqpUVFS6 urrMzMwyMjIQEBCIiIh2dnYiIiLu7u5mZmaYmJgAAABJJ2g3AAAAAXRSTlMAQObYZgAAAbJJREFU OMtjYGDgu8dAEmBbpQCieHYxkAjYmkEk5wcoNxsm/uS/i4t4ABYNPBN97zFdYGC4DuJwNUBF6+Hy jQwMvF+x6FvxgIH1xgMgjayPb/8BmDt+AvEXhC0wxvE9QGK9Apo+5fsKUHnOT0BTfsH1scPMqwAx IhjQ9E2YD/MSI1CEuweuj9UBQvN+BpG6aPpYD9g/gCrkd2BgKjyA8NZKCKVfACKPwfTxgrkMRxny J0DV5XvOvA0x4/ZuENh/Acy5DzOX4T2IyIA4L4BB/wJUeL4CA88W5HA0ByvZrwD3MVBlShskDF1c /AWgwhIgvQ+Q460CpPE/mA12vzhQNiIB7D0GBo4CqDJQFLxfgKQvfQ2I7AcRzCDV3CBJFrByAyAP GrBs/4CEvwHCf7tlwOL+IKt44f7j2gAkmIDK2D9AXAGKPqbvSNaZQIMLFG5gmxHxcBwUbX8YeH5D oy+jBqGNCep+PmD8xSqg6FPsL2Lgk/9+gadfgYFX/ouLex2SdYywcEwpuWvAwICeXiCxr4Al22Cm bwx9fERlQLA+PtL1gfMfz04kkQlE5fdWiB2LyShfAF2IdSlAaCzgAAAAAElFTkSuQmCC "" style=""display:inline;vertical-align:middle;"" /> ⊢ <img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAH8AAAAVBAMAAACd/CwcAAAAMFBMVEX////c3NwiIiJERESqqqqY mJh2dnZmZmYQEBDu7u4yMjJUVFTMzMyIiIi6uroAAAAs73DqAAAAAXRSTlMAQObYZgAAAcRJREFU OMtjYKAY8F0gqCQOr+xpCHUIm1z0/927rR4wsB/AJsl5eXspYwEDw0oIdz1W05sZGNj/MTAswCaX EcDAVxHAwAvxAav+BITRcBbPXyD+zcAQxoApOUsTSOQLMHBANIrUC8DVcMHN4v0INPobA8MMBkzJ FSDGCwaGVxDuhfsPEMG6AcZibmBgYOoE0g4YkuxfQKQ4A0MZRHiCfwDCllwYg38DA+NioE1sBRiS 8uBwmcPAsBTMncdwHhwWRUogoA9Tf37v3SKQwSwLMCTrYRb2gckHDPJIljB4Qf1zX4CBUxvkwAYM SX1YmP0Dh/bu3fsNkKN4BcQEC5AhQKt4PmBI/gezJ0AN4GNg4ECO61OZEBoYgwzxCWgGQCT7QQQL MIW1gBjAQGZqQHhTyRKaDL4Cif0OMC+gSO4HxSI7EJsCMSNQDReSJd5IyYDxFyiZLcCQBAc6yC3p oEQFivKfcCWMC5CSwQlQUkeKRpgkKzAdPAcF5CMGBuH+hQys9r/gipihAcxu/3v3LnA0cyFSGUyS 4fTqcnDy4hDAyCU8GCIz8EiyJhBRZuTik1xJWD/nAnyy0oQN4BbAJ8tIuEi7g0cOAB2DcpoVpUDU AAAAAElFTkSuQmCC "" style=""display:inline;vertical-align:middle;"" />"
1,modus ponens,"2, 3","<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAALcAAAAVBAMAAAD2lGJJAAAAMFBMVEX///9ERETc3NyqqqpUVFS6 urrMzMwyMjIQEBCIiIh2dnYiIiLu7u5mZmaYmJgAAABJJ2g3AAAAAXRSTlMAQObYZgAAAjlJREFU OMu1VT1IHFEQHlfvztPdNaTRQnCDRaqghZDSKxbEIpyIhU3AQlCw8AwKEkKMSFIF7iytFNKoCCrY +AN3TdTO6xREvSK9YhSJmpiZ97dv915xhTew875vdvbbt/PmvQUA9xye3+KrHg32DlTD4nPkk1eC jmm3xkz5v558v7nPRHSz59+dW0WAYyINX0R0RsuYMc5mFsC5NRLNlksQOynhqIu7+YJK0DEoGP+D 142J4HwlONhDt+5FxNsvPJWqY/gkQfIaX3tvIgAJOYVpAoMQEc8tBhXUMQxIUIOZjd9NBCCW4qPz m/ybiHis0F1SmRrGhJwATSmwJgomgrbCh44M+X0p7jAKP2Fciijs7DL7K8LjvQun6qUBOWVJ+SIj Fyrhktwor0AfdBRlXMdotVN8XPTA3pLBECF7y3TyarESKDH8jS++76dfyEbQMO+Ar2xoIU05sxBh /TJN6k9BgzXj3cEhVmaA+owsuYbZyz7y2lLjXS6JYIiQjayRz7JPJclGulnHdDqRyd2ksKj5JteO /0OX7oQo4TXfbWXxNOU6quYNG+gsTEtc8e+RWJU8FXS2dSf2iyTKusQ6Uyewbwha8YA69xHshwAr e6119ijuqKMNjUizRB1d7PP3Xkj8VfYDuG13RTvrKayeeym2R9uN3/MZweG1RtQGk10yPHnGKxfa oXwtvfKTaCga6DedsOUHWJm4W8lRXaroQGfibpXE2Xlub2uRXAVPJSv7E/GN5/6o1j/0P+z2uKtH x+MYAAAAAElFTkSuQmCC "" style=""display:inline;vertical-align:middle;"" /> ⊢"
2,assumption,,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAFMAAAAVBAMAAADInc9CAAAAMFBMVEX////c3NxERESqqqpUVFS6 urrMzMwyMjIQEBCIiIh2dnYiIiLu7u5mZmaYmJgAAABXZE4RAAAAAXRSTlMAQObYZgAAAVZJREFU KM9jYGDgu8dACLCtEgBRPLsYCAO2ZhDJ+QFI8Pl/cXEqxabopMrdB7IMDNdBbK4GEMmhwMBwPwBT JVcfA0MsEK9AKM2fAMQOcBUHoDRPNZDF9RFF6Xygu98vgCutgdLsPUCCtQFFqQYQ7zgAVxoCpe8b AAnGCyhKvzAwiOsgOXECmGL8DnZFAlQpbwEoML7MnLsPLMu7Gwz+Qjj/YHrfg4gMkL85gS4/7YDk c2ZwyHF/hPHZgY5IaQOxmICu4PyHHErHW4EE/wewA0CE+gMGhogEkBjQRL5fyNFTBfIjB1ipGch8 UOiwgNyaD9TD8hnJrZvAocHyB0RegLmVawM0WOULkJwKcTfbT5ACkC5EYAGDla0dEawM4jCPGzCw XmRAVnrpv4tL+QMkpyrCnOzue5kBRSkGSEAXwK2UAatSPqKUgtMrz05ickErmOJbTFzeAgD9sl8j gQWWbAAAAABJRU5ErkJggg== "" style=""display:inline;vertical-align:middle;"" /> ⊢ <img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAEEAAAAPBAMAAABJi79qAAAAMFBMVEX///+qqqru7u4iIiJ2dnbc 3NyIiIjMzMxmZmYQEBC6uroyMjJERERUVFSYmJgAAADLUcuvAAAAAXRSTlMAQObYZgAAAPhJREFU GNNjYGB4+//u3dMLGDABU86NPtYGEGsOAwPHLywqPB8wcHU+ADLYvwHxX4ROGEPbCEj4BwAJtq8M DHxf4Cq4FaCMnSDGShCLZwIDA/MkuAquCxCa4yOIjAAR/BcYWPcoIKx3g1DxG0CkOoiov5bW/AAs 2GwMAvZg5zP0P4DryQ9gYLJF9oM02Ov2AXCBkyBVD5DDYSdIyX8wG2w7yKPvHZBUVDuB

In the previous two demonstrations, where the new antecedent was drawn from the set of assumptions, we can think of *hypothetical reasoning* as a procedure in which we transform an "implicit" assumption to an "explicit" antecedent.  Similarly, *modus ponens* may be used to transform an "explicit" antecedent to an "implicit" assumption.  What is the difference between these two different forms of assumption/antecedent?  Why do we need both forms?  The explicit form is necessary because the implicit form cannot be nested.  For example, one could not precisely express $(A \Rightarrow C) \land (B \Rightarrow C) \Rightarrow [(A \lor B) \Rightarrow C]$ with assumptions alone.  The implicit form is also very important.  The implicit form (with assumptions) is extremely convenient, and necessary in the **Prove-It** framework, for accessing the consequent part of an implication directly and applying logical deductions that arise from that consequent (e.g., consider the role of $B$ as a consequent of $A \Rightarrow B$ in the above examples).

The new antecedent does not need to be one of the pre-existing assumptions, however.  After all, a **KnownTruth** is just as valid when extra assumptions are added (the requirements are simply over-complete).  For example,

In [20]:
X_impl_C = CviaTransitivity.asImplication(X)

X_impl_C: {A , A => B , B => C} |- X => C

In [21]:
X_impl_C.proof()

,step type,requirements,statement
0,hypothetical reasoning,1,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAALcAAAAVBAMAAAD2lGJJAAAAMFBMVEX///9ERETc3NyqqqpUVFS6 urrMzMwyMjIQEBCIiIh2dnYiIiLu7u5mZmaYmJgAAABJJ2g3AAAAAXRSTlMAQObYZgAAAjlJREFU OMu1VT1IHFEQHlfvztPdNaTRQnCDRaqghZDSKxbEIpyIhU3AQlCw8AwKEkKMSFIF7iytFNKoCCrY +AN3TdTO6xREvSK9YhSJmpiZ97dv915xhTew875vdvbbt/PmvQUA9xye3+KrHg32DlTD4nPkk1eC jmm3xkz5v558v7nPRHSz59+dW0WAYyINX0R0RsuYMc5mFsC5NRLNlksQOynhqIu7+YJK0DEoGP+D 142J4HwlONhDt+5FxNsvPJWqY/gkQfIaX3tvIgAJOYVpAoMQEc8tBhXUMQxIUIOZjd9NBCCW4qPz m/ybiHis0F1SmRrGhJwATSmwJgomgrbCh44M+X0p7jAKP2Fciijs7DL7K8LjvQun6qUBOWVJ+SIj Fyrhktwor0AfdBRlXMdotVN8XPTA3pLBECF7y3TyarESKDH8jS++76dfyEbQMO+Ar2xoIU05sxBh /TJN6k9BgzXj3cEhVmaA+owsuYbZyz7y2lLjXS6JYIiQjayRz7JPJclGulnHdDqRyd2ksKj5JteO /0OX7oQo4TXfbWXxNOU6quYNG+gsTEtc8e+RWJU8FXS2dSf2iyTKusQ6Uyewbwha8YA69xHshwAr e6119ijuqKMNjUizRB1d7PP3Xkj8VfYDuG13RTvrKayeeym2R9uN3/MZweG1RtQGk10yPHnGKxfa oXwtvfKTaCga6DedsOUHWJm4W8lRXaroQGfibpXE2Xlub2uRXAVPJSv7E/GN5/6o1j/0P+z2uKtH x+MYAAAAAElFTkSuQmCC "" style=""display:inline;vertical-align:middle;"" /> ⊢ <img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAEMAAAAPBAMAAABNfm9XAAAAMFBMVEX///9ERES6uroiIiKIiIju 7u5mZmbMzMyqqqoyMjLc3Nx2dnYQEBBUVFSYmJgAAADDc6PYAAAAAXRSTlMAQObYZgAAAOlJREFU KM9jYGB4+/80w8o/NxgwAavjtTyuBBDrfgMDxwUsKhj2PGBgynkAYtVPYFiGrBfGiDQFEvsXgJic vyCmQQFnAJQxB8TQATM5vsUiG88EtZTtB4iEmn/+KooTtkCo9RNAZDiEs98BKplsDAL2EGvzHyBp u2+A6pNOBRBpvwAhwrb2E1p4zAGp+Q9mQ9yezv4RVUnlVrALwV4pAJHcCnAzIW4xFoVYDzKADcx0 YmB4jxwuDL0Qqh7kCbB5bBsYGPg/IKngmgCh+YDhogYyXvG8EQPT/M9IIcMI80nt3LQGBqyAHY0P ALfoOsYKQDYqAAAAAElFTkSuQmCC "" style=""display:inline;vertical-align:middle;"" />"
1,modus ponens,"2, 3","<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAALcAAAAVBAMAAAD2lGJJAAAAMFBMVEX///9ERETc3NyqqqpUVFS6 urrMzMwyMjIQEBCIiIh2dnYiIiLu7u5mZmaYmJgAAABJJ2g3AAAAAXRSTlMAQObYZgAAAjlJREFU OMu1VT1IHFEQHlfvztPdNaTRQnCDRaqghZDSKxbEIpyIhU3AQlCw8AwKEkKMSFIF7iytFNKoCCrY +AN3TdTO6xREvSK9YhSJmpiZ97dv915xhTew875vdvbbt/PmvQUA9xye3+KrHg32DlTD4nPkk1eC jmm3xkz5v558v7nPRHSz59+dW0WAYyINX0R0RsuYMc5mFsC5NRLNlksQOynhqIu7+YJK0DEoGP+D 142J4HwlONhDt+5FxNsvPJWqY/gkQfIaX3tvIgAJOYVpAoMQEc8tBhXUMQxIUIOZjd9NBCCW4qPz m/ybiHis0F1SmRrGhJwATSmwJgomgrbCh44M+X0p7jAKP2Fciijs7DL7K8LjvQun6qUBOWVJ+SIj Fyrhktwor0AfdBRlXMdotVN8XPTA3pLBECF7y3TyarESKDH8jS++76dfyEbQMO+Ar2xoIU05sxBh /TJN6k9BgzXj3cEhVmaA+owsuYbZyz7y2lLjXS6JYIiQjayRz7JPJclGulnHdDqRyd2ksKj5JteO /0OX7oQo4TXfbWXxNOU6quYNG+gsTEtc8e+RWJU8FXS2dSf2iyTKusQ6Uyewbwha8YA69xHshwAr e6119ijuqKMNjUizRB1d7PP3Xkj8VfYDuG13RTvrKayeeym2R9uN3/MZweG1RtQGk10yPHnGKxfa oXwtvfKTaCga6DedsOUHWJm4W8lRXaroQGfibpXE2Xlub2uRXAVPJSv7E/GN5/6o1j/0P+z2uKtH x+MYAAAAAElFTkSuQmCC "" style=""display:inline;vertical-align:middle;"" /> ⊢"
2,assumption,,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAFMAAAAVBAMAAADInc9CAAAAMFBMVEX////c3NxERESqqqpUVFS6 urrMzMwyMjIQEBCIiIh2dnYiIiLu7u5mZmaYmJgAAABXZE4RAAAAAXRSTlMAQObYZgAAAVZJREFU KM9jYGDgu8dACLCtEgBRPLsYCAO2ZhDJ+QFI8Pl/cXEqxabopMrdB7IMDNdBbK4GEMmhwMBwPwBT JVcfA0MsEK9AKM2fAMQOcBUHoDRPNZDF9RFF6Xygu98vgCutgdLsPUCCtQFFqQYQ7zgAVxoCpe8b AAnGCyhKvzAwiOsgOXECmGL8DnZFAlQpbwEoML7MnLsPLMu7Gwz+Qjj/YHrfg4gMkL85gS4/7YDk c2ZwyHF/hPHZgY5IaQOxmICu4PyHHErHW4EE/wewA0CE+gMGhogEkBjQRL5fyNFTBfIjB1ipGch8 UOiwgNyaD9TD8hnJrZvAocHyB0RegLmVawM0WOULkJwKcTfbT5ACkC5EYAGDla0dEawM4jCPGzCw XmRAVnrpv4tL+QMkpyrCnOzue5kBRSkGSEAXwK2UAatSPqKUgtMrz05ickErmOJbTFzeAgD9sl8j gQWWbAAAAABJRU5ErkJggg== "" style=""display:inline;vertical-align:middle;"" /> ⊢ <img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAEEAAAAPBAMAAABJi79qAAAAMFBMVEX///+qqqru7u4iIiJ2dnbc 3NyIiIjMzMxmZmYQEBC6uroyMjJERERUVFSYmJgAAADLUcuvAAAAAXRSTlMAQObYZgAAAPhJREFU GNNjYGB4+//u3dMLGDABU86NPtYGEGsOAwPHLywqPB8wcHU+ADLYvwHxX4ROGEPbCEj4BwAJtq8M DHxf4Cq4FaCMnSDGShCLZwIDA/MkuAquCxCa4yOIjAAR/BcYWPcoIKx3g1DxG0CkOoiov5bW/AAs 2GwMAvZg5zP0P4DryQ9gYLJF9oM02Ov2AXCBkyBVD5DDYSdIyX8wG2w7yKPvHZBUVDuByPkggrEA FBz/gMR9AYQ7jA+B1d0HaedggAYH628kI2QhVH0CkHCCBUflJoQC1g0Qmg8YHitAruU4//fu7X1I RvDA/FC7t0OAARtgR+MDAAhbPiS

## Automation regarding `Implies` objects

We will talk more generally about automation in the <a href="tutorial08_automation.ipynb">automation</a> chapter, but in this section we will get a preview of that as we look at automation specific to **implications**.

It is not always necessary to call the `deriveConsequent` method directly.  In fact, the `deriveConsequent` method is called automatically as a "side-effect" whenever a **known truth** for an `Implies` **expression** is created.  In the example below, a **known truth** for $P \Rightarrow Q$ is created via proof-by-assumption which then triggers $Q$ to be derived as a consequent, adding the extra assumption for the antecedent $P$.  Then, a proof for $Q$, under the assumptions of $P \Rightarrow Q$ and $P$, is automatically generated and available upon request:

In [22]:
from proveit._common_ import P, Q, R, S
QByAssumption = Q.prove([P, Implies(P, Q)])

QByAssumption: {P , P => Q} |- Q

In [23]:
QByAssumption.proof()

,step type,requirements,statement
0,modus ponens,"1, 2","<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAGoAAAAVBAMAAABce2awAAAAMFBMVEX///9ERETc3NyqqqpUVFS6 urrMzMwyMjIQEBCIiIh2dnYiIiLu7u5mZmaYmJgAAABJJ2g3AAAAAXRSTlMAQObYZgAAAbRJREFU OMtjYGDgu8dACshaBCJ5djGQBjIMgATnBxBzyn8Xly0KYFFO/y8u3k5YlDOtXeUJpHgLgARXA1jo OwMDYxNElh8oLKGAqWt5AgPLHqgGiC62XwwMzL8gsvkLGBjsF8DU8sAYxwWARAuKLs6vwGD5C5Ge H8DA8P4CTDH7ASijGkTcP4CsixHoO5bPEGl5IPY3gOlidYDQvN9A5PoAZF0gv5wXYIB5kecjwjsr IZQ+WHa9AUQDOEhAfmHbC5EGeTEb7MDbu0FgP8Sx9x+A3ZHAwAcK9IwAsF+8Vt2CBhvnl1VrFyGH nTlYxX6w/34AcS0DQ0obxC8H4IoY/6CFOE9FAFQ9JJgZ6xgYIhJA3N8IRfwb0HSlr4GrYO8BEqDE xALyF9tfhKJ8eKBD/LVbBsypBxH2QCvA/uLaAI0uIDgAiy5kYAKh7AM0p7G1MiDHMsQvPL/RvAhO fQUQmmvPAi5DBWRdZ+W/TADp6ldg4Ln43xNFFyMsRYZUzTZgSEBJUdB0gCXNsiEZkMCHTRcf/pzF eCEcpoGPeF0s/7/CEhPPToTwBPy62DoewBMT32KSyw0AFUN6gVp7F/8AAAAASUVORK5CYII= "" style=""display:inline;vertical-align:middle;"" /> ⊢"
1,assumption,,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAFMAAAAVBAMAAADInc9CAAAAMFBMVEX///9ERETc3NyqqqpUVFS6 urrMzMwyMjIQEBCIiIh2dnYiIiLu7u5mZmaYmJgAAABJJ2g3AAAAAXRSTlMAQObYZgAAAV5JREFU KM9jYGDgu8dACLCtUgBRPLsYCAO2ZhDJ+QFE+H9x8XbCpkjr1pIJQOo6iM3VACL5CxgYJBQwVXKV MTB4H2BgWIFQmr+AgcF+AUwBD5xRCCQ4HFCUzg9gYHh/AaaC/QCMsQFI8P5BUSoPxP4GMKWsDlDG /QSQiq8oSr8DLfuIcOJKCMUEFuL6DFXKWwAKjF8MDNlg+2/vBoH9ELfw/gORjN+AjgMxMgJAgfVl 1dpFyD43B4kycP8Bhw7QXnag3pQ2sMY/aIHEUxEAVQQNGfEHDAwRIJfzb0BTmr6GAS5cAQwPbpBy FpBb8+HhBHHrbhlIWIFikhnoVIhbuTZAgxUZmEAo5q9MMxPAnkEElvwBFJVMBVDG6lMMi4UYkJTy XPzviaKUEZYaWN1XbmZgQzEVPc0hscUYnuFTigxkWQtgSvkIKPX/HwBLrzw78SvN6AO5qBXM5ltM XN4CACBwYWeksR12AAAAAElFTkSuQmCC "" style=""display:inline;vertical-align:middle;"" /> ⊢ <img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAEEAAAASBAMAAAD2w64vAAAAMFBMVEX////u7u5mZmaYmJi6urqq qqp2dnbMzMwQEBBEREQyMjLc3NwiIiJUVFSIiIgAAAAeC3ydAAAAAXRSTlMAQObYZgAAAP5JREFU KM9jYGB4+//u3bMbGDABd1LaPQjrFwMDZzcWFVkFDLwHQQz2bwwMrN/g4gIwhtQEINELYvF9YWBg /gRXsRDGWAMi9EEaOD8wMPB+havwhdJM30FkvgOQ4F/AwCA/Aa6C7QGE3g8Wyg8AEvUJDOyHwLrO gAHUQn0DEDm/AEi8v5amhuxZ1iVg6jzYyT/AygTQfCnVApeD+PEPmgL2ZQJwYa5GkADco1B3nISY uR5ExBdAgwMZsF6A0PEO+56zg+3j/IiqYhfM1wcT2CJBPhCa//cBiorZ8KBb9iaAoQBLfCGJcRYw Y1OBBDgVvPArYOD9/4WACvYOYMgDAPldSOLUB00KAAAAAElFTkSuQmCC "" style=""display:inline;vertical-align:middle;"" />"
2,assumption,,⊢


This can be particularly useful when the proof request is made via some other automation (rather than the manual `prove` request that was demonstrated in the cell above).  This automation is enabled via a `sideEffects` method in the `Implies` class which yields `deriveConsequent` as a method that should be called when an `Implies` object is created.

In [24]:
help(Implies.sideEffects)

Help on function sideEffects in module proveit.logic.boolean.implication.implies:

sideEffects(self, knownTruth)
    Yield the TransitiveRelation side-effects (which also records knownLeftSides
    and knownRightSides).  Also derive the consequent as a side-effect.
    As a special case, if the consequent is FALSE, do deriveViaContradiction.



Note that there are several things that may be attempted as automation here, not just `deriveConsequent`.  In general, the `sideEffects` method of an **Expression** is called, if it exists, whenever a **KnownTruth** for that **Expression** is created.  The **KnownTruth** object is passed to this method and the method should yield methods that should be called for deriving desired side-effects.  This enables automation for a variety of **Expression** types, not just `Implies` objects.  Another way that automation may be performed is by implementing a `conclude` method which may attempt to automatically prove a particular type of expression under a given set of assumptions:

In [25]:
help(Implies.conclude)

Help on function conclude in module proveit.logic.boolean.implication.implies:

conclude(self, assumptions)
    Try to automatically conclude this implication by reducing its operands
    to true/false, or by doing a "transitivity" search amongst known true implications
    whose assumptions are covered by the given assumptions.



The following example of automation relies on the `sideEffects` method to populate a dictionary of **KnownTruth** implications and also on `conclude` to perform a search over these implications to find a path to a conclusion from a hypothesis using intermediate implications via transitivity relations (from $A \Rightarrow B$ and $B \Rightarrow C$, we can obtain $A \Rightarrow C$, as we proved in the previous section). 

In [26]:
P_impl_S = Implies(P, S).prove([Implies(P, Q), Implies(Q, R), Implies(R, S)])

P_impl_S: {P => Q , Q => R , R => S} |- P => S

Below, we display the proof for the above **known truth** that was proven via automation. Notice that the proof relies upon invoking a theorem and applying *specialization* which will be discussed in detail in later tutorial chapters.  Consider this to be a sneak peak.

In [27]:
P_impl_S.proof()

step type	requirements	statement
0	specialization	3, 1, 2	{P => Q , Q => R , R => S} |- P => S
	A : P, B : R, C : S
1	specialization	3, 4, 5	{P => Q , Q => R} |- P => R
	A : P, B : Q, C : R
2	assumption		{R => S} |- R => S
3	theorem		|- forall_{A, B, C | A => B , B => C} (A => C)
	proveit.logic.boolean.implication.implicationTransitivity
4	assumption		{P => Q} |- P => Q
5	assumption		{Q => R} |- Q => R

We can disable this particular proof and obtain an alternate proof via automation as well.

In [28]:
P_impl_S.proof().disable()

Note, below, that we do not need to supply the assumptions because `P_impl_S` is a `KnownTruth` object that automatically includes its own assumptions.

In [29]:
P_impl_S.prove().proof()

step type	requirements	statement
0	hypothetical reasoning	1	{P => Q , Q => R , R => S} |- P => S
1	modus ponens	2, 3	{P => Q , Q => R , R => S , P} |- S
2	assumption		{R => S} |- R => S
3	modus ponens	4, 5	{P => Q , Q => R , P} |- R
4	specialization	6, 7, 8	{P => Q , Q => R} |- P => R
	A : P, B : Q, C : R
5	assumption		{P} |- P
6	theorem		|- forall_{A, B, C | A => B , B => C} (A => C)
	proveit.logic.boolean.implication.implicationTransitivity
7	assumption		{P => Q} |- P => Q
8	assumption		{Q => R} |- Q => R

This is a slightly longer proof but it demonstrates that *hypothetical reasoning* can also be automated.  That is, the `Implies.conclude()` method will attempt to apply `asImplication` automatically if other strategies fail (like the previous transitivity approach, which fails because that proof was disabled).

## Disabling/Enabling Automation

If desired, for whatever reason, automation (via `sideEffects` and `conclude`) may be disabled by setting the `automation` flag of the `defaults` object to `False`:

In [30]:
from proveit import defaults
defaults.automation = False

defaults.automation: False

We now attempt an automated proof through transitivity relations similar to what we performed before. In this case we use the reverse direction, $\{S \Rightarrow R, R \Rightarrow Q, Q \Rightarrow P\} \vdash S \Rightarrow P$, to make it different, otherwise it would remember the solution from before.  The proof will fail because automation is disabled.

In [31]:
from proveit import ProofFailure
try:
    Implies(S, P).prove([Implies(S, R), Implies(R, Q), Implies(Q, P)])
    assert False, "Expecting an ProofFailure error; should not make it to this point"
except ProofFailure as e:
    print("Expected error:", e)

Expected error: Unable to prove S => P assuming {S => R, R => Q, Q => P}: No pre-existing proof


It may be re-enabled by setting this flag back to `True`:

In [32]:
defaults.automation = True

defaults.automation: True

Now the proof will go through via automation:

In [33]:
Implies(S, P).prove([Implies(S, R), Implies(R, Q), Implies(Q, P)])

{S => R , R => Q , Q => P} |- S => P

In addition to changing `defaults.automation`, it is also possible to disable automation for a particular instance when calling `prove` (by supplying the argument `automation=False` as part of the method call).  Basically, this just checks if something has been proven already (or is proven by the assumptions or their automatic side-effects) and raises a `ProofFailure` otherwise.  This can be useful in other automation to quickly check a possible proof pathway without potentially wasting the effort to commit to that pathway.

For example, we can check if there already exists a proof for (or automatic side effect giving) $X \Rightarrow Z$ given the assumptions that $X \Rightarrow Y$ and $Y \Rightarrow Z$:

In [34]:
from proveit._common_ import X, Y, Z
try:
    Implies(X, Z).prove([Implies(X, Y), Implies(Y, Z)], automation=False)
    assert False, "Expecting an ProofFailure error; should not make it to this point"
except ProofFailure as e:
    print("Expected error:", e)

Expected error: Unable to prove X => Z assuming {X => Y, Y => Z}: No pre-existing proof


But using the default `automation=True` (*i.e.*, using the "default" by not even supplying the automation argument), we can automatically prove this implication via transitivity (just as we saw above using different labels).

In [35]:
X_impl_Z = Implies(X, Z).prove([Implies(X, Y), Implies(Y, Z)])

X_impl_Z: {X => Y , Y => Z} |- X => Z

In [36]:
X_impl_Z.proof()

step type	requirements	statement
0	specialization	1, 2, 3	{X => Y , Y => Z} |- X => Z
	A : X, B : Y, C : Z
1	theorem		|- forall_{A, B, C | A => B , B => C} (A => C)
	proveit.logic.boolean.implication.implicationTransitivity
2	assumption		{X => Y} |- X => Y
3	assumption		{Y => Z} |- Y => Z

In [37]:
%end implication

# Next chapter: <a href="tutorial04_relabeling.ipynb">Relabeling</a>

## <a href="tutorial00_introduction.ipynb#contents">Table of Contents</a>